In [55]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import asyncio
import aiohttp
from aiohttp import ClientSession

In [56]:
ta = pd.read_csv("../original_db/title.akas.tsv", sep="\t", na_values="\\N" )
tb = pd.read_csv("../original_db/title.basics.tsv", sep="\t", na_values="\\N" )
tr = pd.read_csv("../original_db/title.ratings.tsv", sep="\t", na_values="\\N" )
nb = pd.read_csv("../original_db/name.basics.tsv", sep="\t", na_values="\\N" )

/Users/julien/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/julien/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


On a récupérer un ensemble de classement directement sur IMDB afin de pouvoir créer une table de films et personnes\
les plus connus possible. 

In [57]:
# Import ranking IMDB
rkg_people = pd.read_excel("../original_db/rkg_people.xlsx")

In [58]:
# Fonction pourcentage de null
def nan_percent(df):
     print(f"{(df.isnull().sum())/len(df.index)*100}% de NaN")


## Merge et clean de toutes les tables utiles: 

In [59]:
# Lower columns
tr.columns = [i.lower() for i in tr.columns]

In [60]:
tb.columns = [i.lower() for i in tb.columns]
tb['runtimeminutes'] =pd.to_numeric(tb['runtimeminutes'], errors='coerce', downcast=None)

In [61]:
tb

,tconst,titletype,primarytitle,originaltitle,isadult,startyear,endyear,runtimeminutes,genres
0,tt0000001,short,Carmencita,Carmencita,0.0,1894.0,NaN,1.0,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0.0,1892.0,NaN,5.0,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0.0,1892.0,NaN,4.0,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0.0,1892.0,NaN,12.0,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0.0,1893.0,NaN,1.0,"Comedy,Short"
...,...,...,...,...,...,...,...,...,...
8821670,tt9916848,tvEpisode,Episode #3.17,Episode #3.17,0.0,2010.0,NaN,NaN,"Action,Drama,Family"
8821671,tt9916850,tvEpisode,Episode #3.19,Episode #3.19,0.0,2010.0,NaN,NaN,"Action,Drama,Family"
8821672,tt9916852,tvEpisode,Episode #3.20,Episode #3.20,0.0,2010.0,NaN,NaN,"Action,Drama,Family"
8821673,tt9916856,short,The Wind,The Wind,0.0,2015.0,NaN,27.0,Short


In [62]:
tb.groupby(by='titletype').count().sort_values(by="tconst", ascending=False)

,tconst,primarytitle,originaltitle,isadult,startyear,endyear,runtimeminutes,genres
titletype,,,,,,,,
tvEpisode,6613989,6613981,6613981,6613988,5573830,0,1038089,6333841
short,863251,863251,863251,863251,838962,0,560478,863102
movie,606294,606294,606294,606294,527547,0,381564,534795
video,258883,258883,258883,258883,257528,0,174990,250673
tvSeries,223017,223017,223017,223017,206740,70299,89027,204343
tvMovie,135603,135603,135603,135603,132080,0,91772,124103
tvMiniSeries,42841,42841,42841,42841,40196,18796,15850,40035
tvSpecial,36467,36467,36467,36467,36301,0,17418,31173
videoGame,30817,30817,30817,30817,30334,0,302,25978


Choix de garder uniquement les titletype movie

In [63]:
# Trie selon methode de François: 
new_tb = tb[(
                (tb['titletype']=='movie') 
            )]

In [64]:
# Nombre de lignes restantes:
new_tb.shape[0]

606294

In [65]:
nan_percent(new_tb)

tconst              0.000000
titletype           0.000000
primarytitle        0.000000
originaltitle       0.000000
isadult             0.000000
startyear          12.988253
endyear           100.000000
runtimeminutes     37.066176
genres             11.792794
dtype: float64% de NaN


**On merge title Rating et title Basic**

In [66]:
trtb = pd.merge(tr ,new_tb, how='inner', left_on='tconst', right_on='tconst')
nan_percent(trtb)

tconst              0.000000
averagerating       0.000000
numvotes            0.000000
titletype           0.000000
primarytitle        0.000000
originaltitle       0.000000
isadult             0.000000
startyear           0.011560
endyear           100.000000
runtimeminutes     10.066724
genres              3.716950
dtype: float64% de NaN


In [67]:
# Lower columns
ta.columns = [i.lower() for i in ta.columns]

**On merge ensuite avec title akas**

In [68]:
# Merge des trois
trtbta = pd.merge(trtb, ta, how='inner', left_on='tconst', right_on='titleid')
trtbta

,tconst,averagerating,numvotes,titletype,primarytitle,originaltitle,isadult,startyear,endyear,runtimeminutes,genres,titleid,ordering,title,region,language,types,attributes,isoriginaltitle
0,tt0000502,4.5,14,movie,Bohemios,Bohemios,0.0,1905.0,NaN,100.0,NaN,tt0000502,1,Bohemios,NaN,NaN,original,NaN,1.0
1,tt0000502,4.5,14,movie,Bohemios,Bohemios,0.0,1905.0,NaN,100.0,NaN,tt0000502,2,Bohemios,ES,NaN,imdbDisplay,NaN,0.0
2,tt0000574,6.0,758,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0.0,1906.0,NaN,70.0,"Action,Adventure,Biography",tt0000574,10,The Story of the Kelly Gang,AU,NaN,imdbDisplay,NaN,0.0
3,tt0000574,6.0,758,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0.0,1906.0,NaN,70.0,"Action,Adventure,Biography",tt0000574,1,Kelly bandájának története,HU,NaN,imdbDisplay,NaN,0.0
4,tt0000574,6.0,758,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0.0,1906.0,NaN,70.0,"Action,Adventure,Biography",tt0000574,2,Ned Kelly and His Gang,AU,NaN,imdbDisplay,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1925381,tt9916428,3.6,15,movie,The Secret of China,Hong xing zhao yao Zhong guo,0.0,2019.0,NaN,NaN,"Adventure,History,War",tt9916428,3,The Secret of China,US,NaN,imdbDisplay,NaN,0.0
1925382,tt9916428,3.6,15,movie,The Secret of China,Hong xing zhao yao Zhong guo,0.0,2019.0,NaN,NaN,"Adventure,History,War",tt9916428,4,The Secret of China,GB,NaN,imdbDisplay,NaN,0.0
1925383,tt9916428,3.6,15,movie,The Secret of China,Hong xing zhao yao Zhong guo,0.0,2019.0,NaN,NaN,"Adventure,History,War",tt9916428,5,Hong xing zhao yao Zhong guo,CN,NaN,NaN,NaN,0.0
1925384,tt9916538,8.3,6,movie,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,0.0,2019.0,NaN,123.0,Drama,tt9916538,1,Kuambil Lagi Hatiku,ID,NaN,NaN,NaN,0.0


**On garde seulement quelques colonnes**

In [69]:
trtbta1 = trtbta[['tconst', 'averagerating', 'numvotes', 'primarytitle', 'originaltitle', 'isadult', 'startyear', 'runtimeminutes', 'types', 'isoriginaltitle', 'genres']]

In [70]:
# Nouvelle table: 
trtbta1

,tconst,averagerating,numvotes,primarytitle,originaltitle,isadult,startyear,runtimeminutes,types,isoriginaltitle,genres
0,tt0000502,4.5,14,Bohemios,Bohemios,0.0,1905.0,100.0,original,1.0,NaN
1,tt0000502,4.5,14,Bohemios,Bohemios,0.0,1905.0,100.0,imdbDisplay,0.0,NaN
2,tt0000574,6.0,758,The Story of the Kelly Gang,The Story of the Kelly Gang,0.0,1906.0,70.0,imdbDisplay,0.0,"Action,Adventure,Biography"
3,tt0000574,6.0,758,The Story of the Kelly Gang,The Story of the Kelly Gang,0.0,1906.0,70.0,imdbDisplay,0.0,"Action,Adventure,Biography"
4,tt0000574,6.0,758,The Story of the Kelly Gang,The Story of the Kelly Gang,0.0,1906.0,70.0,imdbDisplay,0.0,"Action,Adventure,Biography"
...,...,...,...,...,...,...,...,...,...,...,...
1925381,tt9916428,3.6,15,The Secret of China,Hong xing zhao yao Zhong guo,0.0,2019.0,NaN,imdbDisplay,0.0,"Adventure,History,War"
1925382,tt9916428,3.6,15,The Secret of China,Hong xing zhao yao Zhong guo,0.0,2019.0,NaN,imdbDisplay,0.0,"Adventure,History,War"
1925383,tt9916428,3.6,15,The Secret of China,Hong xing zhao yao Zhong guo,0.0,2019.0,NaN,NaN,0.0,"Adventure,History,War"
1925384,tt9916538,8.3,6,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,0.0,2019.0,123.0,NaN,0.0,Drama


In [71]:
trtbta1 = trtbta1[~trtbta1.genres.isna()]

In [72]:
trtbta1

,tconst,averagerating,numvotes,primarytitle,originaltitle,isadult,startyear,runtimeminutes,types,isoriginaltitle,genres
2,tt0000574,6.0,758,The Story of the Kelly Gang,The Story of the Kelly Gang,0.0,1906.0,70.0,imdbDisplay,0.0,"Action,Adventure,Biography"
3,tt0000574,6.0,758,The Story of the Kelly Gang,The Story of the Kelly Gang,0.0,1906.0,70.0,imdbDisplay,0.0,"Action,Adventure,Biography"
4,tt0000574,6.0,758,The Story of the Kelly Gang,The Story of the Kelly Gang,0.0,1906.0,70.0,imdbDisplay,0.0,"Action,Adventure,Biography"
5,tt0000574,6.0,758,The Story of the Kelly Gang,The Story of the Kelly Gang,0.0,1906.0,70.0,imdbDisplay,0.0,"Action,Adventure,Biography"
6,tt0000574,6.0,758,The Story of the Kelly Gang,The Story of the Kelly Gang,0.0,1906.0,70.0,imdbDisplay,0.0,"Action,Adventure,Biography"
...,...,...,...,...,...,...,...,...,...,...,...
1925380,tt9916428,3.6,15,The Secret of China,Hong xing zhao yao Zhong guo,0.0,2019.0,NaN,original,1.0,"Adventure,History,War"
1925381,tt9916428,3.6,15,The Secret of China,Hong xing zhao yao Zhong guo,0.0,2019.0,NaN,imdbDisplay,0.0,"Adventure,History,War"
1925382,tt9916428,3.6,15,The Secret of China,Hong xing zhao yao Zhong guo,0.0,2019.0,NaN,imdbDisplay,0.0,"Adventure,History,War"
1925383,tt9916428,3.6,15,The Secret of China,Hong xing zhao yao Zhong guo,0.0,2019.0,NaN,NaN,0.0,"Adventure,History,War"


In [73]:
trtbta1.drop_duplicates('tconst', keep='first', inplace=True)

/Users/julien/opt/anaconda3/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [74]:
trtbta1.shape[0]

265887

## On utilise la formule de IMDB pour garder que les films les plus populaires. 

weighted rating (𝑊𝑅)=(𝑣/(𝑣+𝑚))𝑅+(𝑚/(𝑣+𝑚))𝐶 \
where:

𝑅 = average for the movie (mean) = (Rating)\
𝑣 = number of votes for the movie = (votes)\
𝑚 = minimum votes required (Quantile 0.5) \
𝐶 = the mean vote across the whole report (Mean)

In [75]:
C = trtbta1.averagerating.mean()
m = trtbta1.numvotes.quantile(.50)
trtbta1 = trtbta1.copy()[trtbta1.numvotes >= m]
def imdb_wr(x, m=m, C=C):
    v = x['numvotes']
    R = x['averagerating']
    return (v/(v+m) * R) + (m/(m+v) * C)
trtbta1['wr_score'] = trtbta1.apply(imdb_wr, axis=1)

In [76]:
trtbta1

,tconst,averagerating,numvotes,primarytitle,originaltitle,isadult,startyear,runtimeminutes,types,isoriginaltitle,genres,wr_score
2,tt0000574,6.0,758,The Story of the Kelly Gang,The Story of the Kelly Gang,0.0,1906.0,70.0,imdbDisplay,0.0,"Action,Adventure,Biography",6.011313
24,tt0000679,5.3,65,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0.0,1908.0,120.0,original,1.0,"Adventure,Fantasy",5.717092
159,tt0001892,5.9,232,Den sorte drøm,Den sorte drøm,0.0,1911.0,53.0,imdbDisplay,0.0,Drama,5.952840
176,tt0001964,6.0,82,The Traitress,Die Verräterin,0.0,1911.0,48.0,original,1.0,Drama,6.064053
201,tt0002101,5.1,530,Cleopatra,Cleopatra,0.0,1912.0,100.0,original,1.0,"Drama,History",5.211278
...,...,...,...,...,...,...,...,...,...,...,...,...
1925306,tt9914644,8.1,99,9/11: Escape from the Towers,9/11: Escape from the Towers,0.0,2018.0,120.0,original,1.0,Documentary,7.340665
1925310,tt9914942,6.7,142,Life Without Sara Amat,La vida sense la Sara Amat,0.0,2019.0,74.0,imdbDisplay,0.0,Drama,6.530184
1925347,tt9916190,3.6,231,Safeguard,Safeguard,0.0,2020.0,90.0,imdbDisplay,0.0,"Action,Adventure,Thriller",4.145877
1925353,tt9916270,5.8,1327,Il talento del calabrone,Il talento del calabrone,0.0,2020.0,84.0,imdbDisplay,0.0,Thriller,5.815747


## On utilise la table Name Basic

In [77]:
nb

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899.0,1987.0,"soundtrack,actor,miscellaneous","tt0072308,tt0031983,tt0050419,tt0053137"
1,nm0000002,Lauren Bacall,1924.0,2014.0,"actress,soundtrack","tt0037382,tt0038355,tt0117057,tt0071877"
2,nm0000003,Brigitte Bardot,1934.0,NaN,"actress,soundtrack,music_department","tt0057345,tt0049189,tt0054452,tt0056404"
3,nm0000004,John Belushi,1949.0,1982.0,"actor,soundtrack,writer","tt0078723,tt0072562,tt0080455,tt0077975"
4,nm0000005,Ingmar Bergman,1918.0,2007.0,"writer,director,actor","tt0069467,tt0060827,tt0083922,tt0050986"
...,...,...,...,...,...,...
11528988,nm9993714,Romeo del Rosario,NaN,NaN,"animation_department,art_department","tt2455546,tt14069590,tt11657662"
11528989,nm9993716,Essias Loberg,NaN,NaN,NaN,NaN
11528990,nm9993717,Harikrishnan Rajan,NaN,NaN,cinematographer,tt8736744
11528991,nm9993718,Aayush Nair,NaN,NaN,cinematographer,NaN


In [78]:
nb.columns = [i.lower() for i in nb.columns]

In [79]:
nb = nb[~nb.knownfortitles.isna()]

In [80]:
nb.knownfortitles = nb.knownfortitles.apply(lambda x: x.split(','))

/Users/julien/opt/anaconda3/lib/python3.9/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [81]:
kft = nb.explode('knownfortitles')

In [82]:
kft

,nconst,primaryname,birthyear,deathyear,primaryprofession,knownfortitles
0,nm0000001,Fred Astaire,1899.0,1987.0,"soundtrack,actor,miscellaneous",tt0072308
0,nm0000001,Fred Astaire,1899.0,1987.0,"soundtrack,actor,miscellaneous",tt0031983
0,nm0000001,Fred Astaire,1899.0,1987.0,"soundtrack,actor,miscellaneous",tt0050419
0,nm0000001,Fred Astaire,1899.0,1987.0,"soundtrack,actor,miscellaneous",tt0053137
1,nm0000002,Lauren Bacall,1924.0,2014.0,"actress,soundtrack",tt0037382
...,...,...,...,...,...,...
11528987,nm9993713,Sambit Mishra,NaN,NaN,writer,tt10709066
11528988,nm9993714,Romeo del Rosario,NaN,NaN,"animation_department,art_department",tt2455546
11528988,nm9993714,Romeo del Rosario,NaN,NaN,"animation_department,art_department",tt14069590
11528988,nm9993714,Romeo del Rosario,NaN,NaN,"animation_department,art_department",tt11657662


In [83]:
trtbta1

,tconst,averagerating,numvotes,primarytitle,originaltitle,isadult,startyear,runtimeminutes,types,isoriginaltitle,genres,wr_score
2,tt0000574,6.0,758,The Story of the Kelly Gang,The Story of the Kelly Gang,0.0,1906.0,70.0,imdbDisplay,0.0,"Action,Adventure,Biography",6.011313
24,tt0000679,5.3,65,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0.0,1908.0,120.0,original,1.0,"Adventure,Fantasy",5.717092
159,tt0001892,5.9,232,Den sorte drøm,Den sorte drøm,0.0,1911.0,53.0,imdbDisplay,0.0,Drama,5.952840
176,tt0001964,6.0,82,The Traitress,Die Verräterin,0.0,1911.0,48.0,original,1.0,Drama,6.064053
201,tt0002101,5.1,530,Cleopatra,Cleopatra,0.0,1912.0,100.0,original,1.0,"Drama,History",5.211278
...,...,...,...,...,...,...,...,...,...,...,...,...
1925306,tt9914644,8.1,99,9/11: Escape from the Towers,9/11: Escape from the Towers,0.0,2018.0,120.0,original,1.0,Documentary,7.340665
1925310,tt9914942,6.7,142,Life Without Sara Amat,La vida sense la Sara Amat,0.0,2019.0,74.0,imdbDisplay,0.0,Drama,6.530184
1925347,tt9916190,3.6,231,Safeguard,Safeguard,0.0,2020.0,90.0,imdbDisplay,0.0,"Action,Adventure,Thriller",4.145877
1925353,tt9916270,5.8,1327,Il talento del calabrone,Il talento del calabrone,0.0,2020.0,84.0,imdbDisplay,0.0,Thriller,5.815747


**On merge toutes les tables clean**

In [84]:
new_table = kft.merge(trtbta1, how='inner', left_on='knownfortitles', right_on='tconst')

In [85]:
new_table

,nconst,primaryname,birthyear,deathyear,primaryprofession,knownfortitles,tconst,averagerating,numvotes,primarytitle,originaltitle,isadult,startyear,runtimeminutes,types,isoriginaltitle,genres,wr_score
0,nm0000001,Fred Astaire,1899.0,1987.0,"soundtrack,actor,miscellaneous",tt0072308,tt0072308,7.0,42983,The Towering Inferno,The Towering Inferno,0.0,1974.0,165.0,alternative,0.0,"Action,Drama,Thriller",6.998752
1,nm0000328,Richard Chamberlain,1934.0,NaN,"actor,soundtrack,producer",tt0072308,tt0072308,7.0,42983,The Towering Inferno,The Towering Inferno,0.0,1974.0,165.0,alternative,0.0,"Action,Drama,Thriller",6.998752
2,nm0000740,Irwin Allen,1916.0,1991.0,"writer,producer,director",tt0072308,tt0072308,7.0,42983,The Towering Inferno,The Towering Inferno,0.0,1974.0,165.0,alternative,0.0,"Action,Drama,Thriller",6.998752
3,nm0001740,O.J. Simpson,1947.0,NaN,"actor,producer",tt0072308,tt0072308,7.0,42983,The Towering Inferno,The Towering Inferno,0.0,1974.0,165.0,alternative,0.0,"Action,Drama,Thriller",6.998752
4,nm0001816,Robert Vaughn,1932.0,2016.0,"actor,director",tt0072308,tt0072308,7.0,42983,The Towering Inferno,The Towering Inferno,0.0,1974.0,165.0,alternative,0.0,"Action,Drama,Thriller",6.998752
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4660343,nm9987091,A.P. Sathyan,NaN,NaN,assistant_director,tt0292023,tt0292023,7.8,720,Irupatham Noottandu,Irupatham Noottandu,0.0,1987.0,140.0,original,1.0,Drama,7.667034
4660344,nm9987092,G. Ramamoorthy,NaN,NaN,camera_department,tt0292023,tt0292023,7.8,720,Irupatham Noottandu,Irupatham Noottandu,0.0,1987.0,140.0,original,1.0,Drama,7.667034
4660345,nm9987093,S.P. Alagappan,NaN,NaN,camera_department,tt0292023,tt0292023,7.8,720,Irupatham Noottandu,Irupatham Noottandu,0.0,1987.0,140.0,original,1.0,Drama,7.667034
4660346,nm9987094,Karumam Mohan,NaN,NaN,make_up_department,tt0292023,tt0292023,7.8,720,Irupatham Noottandu,Irupatham Noottandu,0.0,1987.0,140.0,original,1.0,Drama,7.667034


## On utilise le fichier excel créer grace à IMDB

Pour rappel: 

Top Acteurs\
Top Actrice\
Top Directors/producers\
Top Writers\
Top acteurs et actrices français\
Top Directors Français


In [86]:
rkg_people = rkg_people.drop_duplicates('Const')

In [87]:
rkg_people

,Const
0,nm0000631
1,nm0003506
2,nm0898288
3,nm0000520
4,nm0000941
...,...
889,nm0920074
890,nm0936404
891,nm0939992
892,nm0943758


In [88]:
new_table = new_table[new_table.nconst.isin(rkg_people.Const)]

In [89]:
new_table

,nconst,primaryname,birthyear,deathyear,primaryprofession,knownfortitles,tconst,averagerating,numvotes,primarytitle,originaltitle,isadult,startyear,runtimeminutes,types,isoriginaltitle,genres,wr_score
216,nm0000007,Humphrey Bogart,1899.0,1957.0,"actor,soundtrack,producer",tt0037382,tt0037382,7.8,34046,To Have and Have Not,To Have and Have Not,0.0,1944.0,100.0,imdbDisplay,0.0,"Adventure,Comedy,Film-Noir",7.796948
269,nm0001328,Howard Hawks,1896.0,1977.0,"director,producer,writer",tt0038355,tt0038355,7.9,84259,The Big Sleep,The Big Sleep,0.0,1946.0,114.0,imdbDisplay,0.0,"Crime,Film-Noir,Mystery",7.898691
357,nm0001215,Albert Finney,1936.0,2019.0,"actor,producer,soundtrack",tt0071877,tt0071877,7.3,61173,Murder on the Orient Express,Murder on the Orient Express,0.0,1974.0,128.0,alternative,0.0,"Crime,Drama,Mystery",7.298814
391,nm0000003,Brigitte Bardot,1934.0,NaN,"actress,soundtrack,music_department",tt0057345,tt0057345,7.5,31387,Contempt,Le mépris,0.0,1963.0,103.0,imdbDisplay,0.0,"Drama,Romance",7.497291
392,nm0000419,Jean-Luc Godard,1930.0,NaN,"director,writer,editor",tt0057345,tt0057345,7.5,31387,Contempt,Le mépris,0.0,1963.0,103.0,imdbDisplay,0.0,"Drama,Romance",7.497291
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4191398,nm1460782,François Damiens,1973.0,NaN,"actor,director,soundtrack",tt8398768,tt8398768,5.8,631,Dany,Mon ket,0.0,2018.0,89.0,imdbDisplay,0.0,"Comedy,Drama",5.831539
4206470,nm1024183,Stéphane De Groodt,1966.0,NaN,"actor,director,writer",tt10143308,tt10143308,5.1,111,Tout nous sourit,Tout nous sourit,0.0,2020.0,101.0,imdbDisplay,0.0,Comedy,5.479240
4254710,nm1812637,Virginie Efira,1977.0,NaN,"actress,writer,producer",tt9173264,tt9173264,5.8,3165,Sibyl,Sibyl,0.0,2019.0,100.0,imdbDisplay,0.0,Drama,5.806781
4394535,nm2650819,Adèle Exarchopoulos,1993.0,NaN,actress,tt4428762,tt4428762,5.9,1823,Down by Love,Éperdument,0.0,2016.0,110.0,imdbDisplay,0.0,"Biography,Drama,Romance",5.908265


In [90]:
nan_percent(new_table)

nconst                0.000000
primaryname           0.000000
birthyear             0.864865
deathyear            70.126126
primaryprofession     0.000000
knownfortitles        0.000000
tconst                0.000000
averagerating         0.000000
numvotes              0.000000
primarytitle          0.000000
originaltitle         0.000000
isadult               0.000000
startyear             0.000000
runtimeminutes        0.000000
types                18.810811
isoriginaltitle       0.000000
genres                0.000000
wr_score              0.000000
dtype: float64% de NaN


In [91]:
new_table = new_table[~new_table.birthyear.isna()]

In [92]:
new_table.describe()

,birthyear,deathyear,averagerating,numvotes,isadult,startyear,runtimeminutes,isoriginaltitle,wr_score
count,2751.000000,829.000000,2751.000000,2.751000e+03,2751.0,2751.000000,2751.000000,2751.000000,2751.000000
mean,1948.959651,1992.471653,7.253581,2.448184e+05,0.0,1991.213740,120.166485,0.028717,7.247836
std,26.767881,20.793631,0.833250,3.745565e+05,0.0,23.419124,28.782089,0.167040,0.830846
min,1873.000000,1925.000000,2.500000,6.300000e+01,0.0,1915.000000,45.000000,0.000000,2.502881
25%,1931.000000,1980.000000,6.800000,2.639300e+04,0.0,1977.000000,102.000000,0.000000,6.798396
50%,1954.000000,1994.000000,7.300000,9.226900e+04,0.0,1999.000000,116.000000,0.000000,7.299498
75%,1970.000000,2010.000000,7.800000,2.936145e+05,0.0,2009.000000,131.000000,0.000000,7.799698
max,1998.000000,2022.000000,9.300000,2.569002e+06,0.0,2021.000000,489.000000,1.000000,9.299923


In [93]:
new_table.drop_duplicates(['tconst', 'nconst'])

,nconst,primaryname,birthyear,deathyear,primaryprofession,knownfortitles,tconst,averagerating,numvotes,primarytitle,originaltitle,isadult,startyear,runtimeminutes,types,isoriginaltitle,genres,wr_score
216,nm0000007,Humphrey Bogart,1899.0,1957.0,"actor,soundtrack,producer",tt0037382,tt0037382,7.8,34046,To Have and Have Not,To Have and Have Not,0.0,1944.0,100.0,imdbDisplay,0.0,"Adventure,Comedy,Film-Noir",7.796948
269,nm0001328,Howard Hawks,1896.0,1977.0,"director,producer,writer",tt0038355,tt0038355,7.9,84259,The Big Sleep,The Big Sleep,0.0,1946.0,114.0,imdbDisplay,0.0,"Crime,Film-Noir,Mystery",7.898691
357,nm0001215,Albert Finney,1936.0,2019.0,"actor,producer,soundtrack",tt0071877,tt0071877,7.3,61173,Murder on the Orient Express,Murder on the Orient Express,0.0,1974.0,128.0,alternative,0.0,"Crime,Drama,Mystery",7.298814
391,nm0000003,Brigitte Bardot,1934.0,NaN,"actress,soundtrack,music_department",tt0057345,tt0057345,7.5,31387,Contempt,Le mépris,0.0,1963.0,103.0,imdbDisplay,0.0,"Drama,Romance",7.497291
392,nm0000419,Jean-Luc Godard,1930.0,NaN,"director,writer,editor",tt0057345,tt0057345,7.5,31387,Contempt,Le mépris,0.0,1963.0,103.0,imdbDisplay,0.0,"Drama,Romance",7.497291
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4005849,nm0903049,Josef von Sternberg,1894.0,1969.0,"director,writer,assistant_director",tt0046712,tt0046712,7.0,1182,Anatahan,Anatahan,0.0,1953.0,91.0,imdbDisplay,0.0,"Drama,History,War",6.956858
4191398,nm1460782,François Damiens,1973.0,NaN,"actor,director,soundtrack",tt8398768,tt8398768,5.8,631,Dany,Mon ket,0.0,2018.0,89.0,imdbDisplay,0.0,"Comedy,Drama",5.831539
4206470,nm1024183,Stéphane De Groodt,1966.0,NaN,"actor,director,writer",tt10143308,tt10143308,5.1,111,Tout nous sourit,Tout nous sourit,0.0,2020.0,101.0,imdbDisplay,0.0,Comedy,5.479240
4254710,nm1812637,Virginie Efira,1977.0,NaN,"actress,writer,producer",tt9173264,tt9173264,5.8,3165,Sibyl,Sibyl,0.0,2019.0,100.0,imdbDisplay,0.0,Drama,5.806781


In [94]:
top_movies = new_table.copy()

## Récupération des données manquantes par API

In [95]:
api_key = 'f68d78c330babd6aae4a2cfde1eb0be6'

In [96]:
async def get_tconst(session, url, tc):
    async with session.get(url) as resp:
        result = await resp.json()
        try:
            title = result['movie_results'][0]['title']
            poster = result['movie_results'][0]['poster_path']
            overview = result['movie_results'][0]['overview']
            olang = result['movie_results'][0]['original_language']
            return {'tc': tc,
                    'title': title, 
                    'poster': f"https://image.tmdb.org/t/p/original{poster}", 
                    'overview': overview,
                    'olang' : olang}
        except:
            pass

async def get_tconsts_infos(tconsts):

    async with aiohttp.ClientSession() as session:

        tasks = []
        for tc in tconsts:
            url = f'https://api.themoviedb.org/3/find/{ tc }?api_key={ api_key }&external_source=imdb_id&language=fr-FR'
            tasks.append(asyncio.ensure_future(get_tconst(session, url, tc)))

        movies = {}
        original_tconst = await asyncio.gather(*tasks)
        for tc in original_tconst:
            try:
                movies[tc['tc']] = tc
            except:
                pass
        return movies

In [97]:
movies_infos = await get_tconsts_infos(top_movies.tconst)

In [98]:
def get_title(x):
    try:
        return movies_infos[x]['title']
    except:
        pass

In [99]:
def get_overview(x):
    try:
        return movies_infos[x]['overview']
    except:
        pass

In [100]:
def get_olang(x):
    try:
        return movies_infos[x]['olang']
    except:
        pass

In [101]:
top_movies['french_title'] = top_movies.tconst.apply(lambda x: get_title(x))
top_movies['overview'] = top_movies.tconst.apply(lambda x: get_overview(x))
top_movies['original_language'] = top_movies.tconst.apply(lambda x: get_olang(x))

In [102]:
top_movies.groupby('original_language').count().sort_values('nconst', ascending=False).head(7)

,nconst,primaryname,birthyear,deathyear,primaryprofession,knownfortitles,tconst,averagerating,numvotes,primarytitle,originaltitle,isadult,startyear,runtimeminutes,types,isoriginaltitle,genres,wr_score,french_title,overview
original_language,,,,,,,,,,,,,,,,,,,,
en,2045,2045,2045,545,2045,2045,2045,2045,2045,2045,2045,2045,2045,2045,1661,2045,2045,2045,2045,2045
fr,556,556,556,225,556,556,556,556,556,556,556,556,556,556,457,556,556,556,556,556
it,27,27,27,21,27,27,27,27,27,27,27,27,27,27,19,27,27,27,27,27
de,27,27,27,10,27,27,27,27,27,27,27,27,27,27,22,27,27,27,27,27
sv,20,20,20,11,20,20,20,20,20,20,20,20,20,20,15,20,20,20,20,20
es,18,18,18,2,18,18,18,18,18,18,18,18,18,18,14,18,18,18,18,18
ja,17,17,17,5,17,17,17,17,17,17,17,17,17,17,12,17,17,17,17,17


In [103]:
top_movies.head()

,nconst,primaryname,birthyear,deathyear,primaryprofession,knownfortitles,tconst,averagerating,numvotes,primarytitle,...,isadult,startyear,runtimeminutes,types,isoriginaltitle,genres,wr_score,french_title,overview,original_language
216,nm0000007,Humphrey Bogart,1899.0,1957.0,"actor,soundtrack,producer",tt0037382,tt0037382,7.8,34046,To Have and Have Not,...,0.0,1944.0,100.0,imdbDisplay,0.0,"Adventure,Comedy,Film-Noir",7.796948,Le Port de l'angoisse,En 1940 à Fort-de-France en Martinique durant ...,en
269,nm0001328,Howard Hawks,1896.0,1977.0,"director,producer,writer",tt0038355,tt0038355,7.9,84259,The Big Sleep,...,0.0,1946.0,114.0,imdbDisplay,0.0,"Crime,Film-Noir,Mystery",7.898691,Le Grand Sommeil,Le général Sternwood engage le détective privé...,en
357,nm0001215,Albert Finney,1936.0,2019.0,"actor,producer,soundtrack",tt0071877,tt0071877,7.3,61173,Murder on the Orient Express,...,0.0,1974.0,128.0,alternative,0.0,"Crime,Drama,Mystery",7.298814,Le Crime de l'Orient-Express,"En décembre 1935, le célèbre train de l'Orient...",en
391,nm0000003,Brigitte Bardot,1934.0,NaN,"actress,soundtrack,music_department",tt0057345,tt0057345,7.5,31387,Contempt,...,0.0,1963.0,103.0,imdbDisplay,0.0,"Drama,Romance",7.497291,Le Mépris,Le scénariste Paul Javal mène une vie heureuse...,fr
392,nm0000419,Jean-Luc Godard,1930.0,NaN,"director,writer,editor",tt0057345,tt0057345,7.5,31387,Contempt,...,0.0,1963.0,103.0,imdbDisplay,0.0,"Drama,Romance",7.497291,Le Mépris,Le scénariste Paul Javal mène une vie heureuse...,fr


## Préparation du fichier à l'export

In [104]:
top_movies.rename(columns={'primaryname': 'name',
                           'birthyear': 'byear',
                           'primaryprofession': 'job',
                           'original_language': 'original_lang',
                           'averagerating': 'averagerating'
                           }, inplace=True)
top_movies = top_movies[['tconst', 'nconst', 'name', 'french_title', 'byear', 'job', 'startyear','runtimeminutes', 'averagerating', 'numvotes','wr_score', 'genres', 'original_lang']]

In [105]:
# Export
top_movies.to_csv('../movies/top_movies.tsv', sep='|')

In [106]:
top_movies

,tconst,nconst,name,french_title,byear,job,startyear,runtimeminutes,averagerating,numvotes,wr_score,genres,original_lang
216,tt0037382,nm0000007,Humphrey Bogart,Le Port de l'angoisse,1899.0,"actor,soundtrack,producer",1944.0,100.0,7.8,34046,7.796948,"Adventure,Comedy,Film-Noir",en
269,tt0038355,nm0001328,Howard Hawks,Le Grand Sommeil,1896.0,"director,producer,writer",1946.0,114.0,7.9,84259,7.898691,"Crime,Film-Noir,Mystery",en
357,tt0071877,nm0001215,Albert Finney,Le Crime de l'Orient-Express,1936.0,"actor,producer,soundtrack",1974.0,128.0,7.3,61173,7.298814,"Crime,Drama,Mystery",en
391,tt0057345,nm0000003,Brigitte Bardot,Le Mépris,1934.0,"actress,soundtrack,music_department",1963.0,103.0,7.5,31387,7.497291,"Drama,Romance",fr
392,tt0057345,nm0000419,Jean-Luc Godard,Le Mépris,1930.0,"director,writer,editor",1963.0,103.0,7.5,31387,7.497291,"Drama,Romance",fr
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4005849,tt0046712,nm0903049,Josef von Sternberg,Anatahan,1894.0,"director,writer,assistant_director",1953.0,91.0,7.0,1182,6.956858,"Drama,History,War",ja
4191398,tt8398768,nm1460782,François Damiens,Mon Ket,1973.0,"actor,director,soundtrack",2018.0,89.0,5.8,631,5.831539,"Comedy,Drama",fr
4206470,tt10143308,nm1024183,Stéphane De Groodt,Tout nous sourit,1966.0,"actor,director,writer",2020.0,101.0,5.1,111,5.479240,Comedy,fr
4254710,tt9173264,nm1812637,Virginie Efira,Sibyl,1977.0,"actress,writer,producer",2019.0,100.0,5.8,3165,5.806781,Drama,fr


In [107]:
top_movies.drop_duplicates('tconst')

,tconst,nconst,name,french_title,byear,job,startyear,runtimeminutes,averagerating,numvotes,wr_score,genres,original_lang
216,tt0037382,nm0000007,Humphrey Bogart,Le Port de l'angoisse,1899.0,"actor,soundtrack,producer",1944.0,100.0,7.8,34046,7.796948,"Adventure,Comedy,Film-Noir",en
269,tt0038355,nm0001328,Howard Hawks,Le Grand Sommeil,1896.0,"director,producer,writer",1946.0,114.0,7.9,84259,7.898691,"Crime,Film-Noir,Mystery",en
357,tt0071877,nm0001215,Albert Finney,Le Crime de l'Orient-Express,1936.0,"actor,producer,soundtrack",1974.0,128.0,7.3,61173,7.298814,"Crime,Drama,Mystery",en
391,tt0057345,nm0000003,Brigitte Bardot,Le Mépris,1934.0,"actress,soundtrack,music_department",1963.0,103.0,7.5,31387,7.497291,"Drama,Romance",fr
406,tt0049189,nm0000003,Brigitte Bardot,Et Dieu… créa la femme,1934.0,"actress,soundtrack,music_department",1956.0,95.0,6.3,8305,6.298851,"Drama,Romance",fr
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4005849,tt0046712,nm0903049,Josef von Sternberg,Anatahan,1894.0,"director,writer,assistant_director",1953.0,91.0,7.0,1182,6.956858,"Drama,History,War",ja
4191398,tt8398768,nm1460782,François Damiens,Mon Ket,1973.0,"actor,director,soundtrack",2018.0,89.0,5.8,631,5.831539,"Comedy,Drama",fr
4206470,tt10143308,nm1024183,Stéphane De Groodt,Tout nous sourit,1966.0,"actor,director,writer",2020.0,101.0,5.1,111,5.479240,Comedy,fr
4254710,tt9173264,nm1812637,Virginie Efira,Sibyl,1977.0,"actress,writer,producer",2019.0,100.0,5.8,3165,5.806781,Drama,fr
